In [1]:
from pyxdf import load_xdf
import numpy as np

The XDF file loaded below was recorded using a [Brain Products actiCHamp](https://github.com/brain-products/LSL-actiCHamp). We sent TTL pulses to the actiCHamp using an [RTBox](https://github.com/xiangruili/RTBox_py), which returned the timestamp that the pulse was actually sent reconciled to the local LSL clock -- those timestamps were recorded in a [marker stream](https://github.com/labstreaminglayer/liblsl-Python). The actiCHamp stream and the marker stream originated from the same computer, and were saved with [LabRecorder](https://github.com/labstreaminglayer/App-LabRecorder).

In [2]:
data, header = load_xdf('sub-P001_ses-S001_task-Default_run-001_eeg.xdf')

In [3]:
type_dict = {data[i]['info']['type'][0]: i for i in range(len(data))}
EEG = type_dict['EEG']
MARKER = type_dict['Markers']

The actiCHamp stream included a marker channel at index 1, which represents the 8-bit trigger values of the TTL pulse as a floating point.

In [4]:
hardware_events = data[EEG]['time_stamps'][data[EEG]['time_series'][:,1] == 255.]
software_events = data[MARKER]['time_stamps']

In [5]:
software_events = software_events[1:] # an event was dropped from EEG for some reason

The timestamps provided by RTBox in the marker stream serve as the "real" event time, which allows us to calculate the lag between the event time and when the event is registered by the actiCHamp client -- the delay of the actiCHamp system.

In [6]:
lags = hardware_events - software_events

In [7]:
np.mean(lags * 1e3) # lag in milliseconds

2.4561026087607702

In [8]:
np.std(lags * 1e3) # jitter in milliseconds

0.031924091819245284